In [1]:
import numpy as np
import pandas as pd
import csv
from sklearn.tree import DecisionTreeClassifier

In [2]:
training_data = pd.read_csv("train70.csv")
testing_data = pd.read_csv("test30.csv")

C:\Users\mathe\AppData\Local\Temp\ipykernel_19440\2947108341.py:1: DtypeWarning: Columns (3,14,19,23) have mixed types. Specify dtype option on import or set low_memory=False.
  training_data = pd.read_csv("train70.csv")
C:\Users\mathe\AppData\Local\Temp\ipykernel_19440\2947108341.py:2: DtypeWarning: Columns (3,14,19,23) have mixed types. Specify dtype option on import or set low_memory=False.
  testing_data = pd.read_csv("test30.csv")


In [3]:
training = pd.DataFrame(training_data)
testing = pd.DataFrame(testing_data)

In [ ]:
# divisão em duas classes

for index, row in training.iterrows():
    if row['target'] == 'dos':
        training.at[index, 'target'] = '1'
    else:
        training.at[index, 'target'] = '0'

for index, row in testing.iterrows():
    if row['target'] == 'dos':
        testing.at[index, 'target'] = '1'
    else:
        testing.at[index, 'target'] = '0'
        
print("Training target: {}".format(training['target'].unique()))
print("Testing target: {}".format(testing['target'].unique()))

In [ ]:
# remoção de features com valores 0

training = training.drop(columns = ['mqtt.conack.flags.reserved', 'mqtt.conack.flags.sp', 'mqtt.conflag.qos', 'mqtt.conflag.reserved', 'mqtt.conflag.retain', 
             'mqtt.conflag.willflag', 'mqtt.retain', 'mqtt.sub.qos', 'mqtt.suback.qos', 'mqtt.willmsg', 'mqtt.willmsg_len', 
             'mqtt.willtopic', 'mqtt.willtopic_len', 'mqtt.msg'])

testing = testing.drop(columns = ['mqtt.conack.flags.reserved', 'mqtt.conack.flags.sp', 'mqtt.conflag.qos', 'mqtt.conflag.reserved', 'mqtt.conflag.retain', 
             'mqtt.conflag.willflag', 'mqtt.retain', 'mqtt.sub.qos', 'mqtt.suback.qos', 'mqtt.willmsg', 'mqtt.willmsg_len', 
             'mqtt.willtopic', 'mqtt.willtopic_len', 'mqtt.msg'])

print(training.shape)
print(testing.shape)

In [ ]:
# ajuste das variaveis categoricas

features_training = pd.get_dummies(training.iloc[:,:-1])
target_training = training['target']

features_testing = pd.get_dummies(testing.iloc[:,:-1])
target_testing = testing['target']

In [ ]:
print(features_training.shape)
print(features_testing.shape)
print(target_training.shape)
print(target_testing.shape)

In [ ]:
col = ['tcp.flags_ACK', 'tcp.flags_FIN ACK', 'tcp.flags_PSH ACK', 'tcp.flags_RST', 'tcp.flags_RST ACK', 'tcp.flags_SYN', 'tcp.flags_SYN ACK', 'tcp.flags_ZERO', 'mqtt.conack.flags_0', 'mqtt.conack.flags_0x00000000', 'mqtt.hdrflags_0', 'mqtt.hdrflags_10000', 'mqtt.hdrflags_100000', 'mqtt.hdrflags_1000000', 'mqtt.hdrflags_10000010', 'mqtt.hdrflags_10010000', 'mqtt.hdrflags_1010000', 'mqtt.hdrflags_110000', 'mqtt.hdrflags_11000000', 'mqtt.hdrflags_110001', 'mqtt.hdrflags_110010', 'mqtt.hdrflags_11010000', 'mqtt.hdrflags_111010', 'mqtt.protoname_0', 'mqtt.protoname_MQTT']

In [ ]:
# substituição das colunas com valores FALSE e TRUE por 0 e 1, respectivamente

features_training[col] = features_training[col].replace({True: 1, False: 0})
features_testing[col] = features_testing[col].replace({True: 1, False: 0})

In [ ]:
features_training = features_training.values
features_testing = features_testing.values
target_training = target_training.values
target_testing = target_testing.values

Decision Tree Classifier

In [ ]:
tree = DecisionTreeClassifier(random_state=0)
tree.fit(features_training, target_training)

In [ ]:
print("Accuracy: {}".format(tree.score(features_training, target_training)))
print("Accuracy: {}".format(tree.score(features_testing, target_testing)))

In [ ]:
predict = tree.predict(features_testing)

In [ ]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(target_testing, predict)

In [ ]:
print("Matriz de Confusão:")
print(conf_matrix)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
%matplotlib inline

In [ ]:
titles_options = [
    ("Confusion matrix, without normalization", None),
    ("Normalized confusion matrix", "true"),
]
for title, normalize in titles_options:
    disp = ConfusionMatrixDisplay.from_estimator(
        tree,
        features_testing,
        target_testing,
        display_labels=training_data['target'],
        cmap=plt.cm.Blues,
        normalize=normalize,
    )
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

plt.show()